In [1]:
!pip install flask_marshmallow flask_sqlalchemy
!pip install marshmallow marshmallow-sqlalchemy

  Obtaining dependency information for flask_marshmallow from https://files.pythonhosted.org/packages/27/cd/5b3d4103460766eb834f6deaf26a87a4735a0c9687f24163e07d1f3d7791/flask_marshmallow-1.3.0-py3-none-any.whl.metadata



[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: C:\Users\bayram\AppData\Local\Programs\Python\Python311\python.exe -m pip install --upgrade pip


  Obtaining dependency information for marshmallow-sqlalchemy from https://files.pythonhosted.org/packages/e4/e4/6cef9deaff67726be477795e85eaadadc3c5e6e661059adfdc8967d04fe2/marshmallow_sqlalchemy-1.1.1-py3-none-any.whl.metadata



[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: C:\Users\bayram\AppData\Local\Programs\Python\Python311\python.exe -m pip install --upgrade pip


In [20]:
from flask import Flask, request, jsonify
from flask_sqlalchemy import SQLAlchemy
from marshmallow import Schema, fields, validate, ValidationError
from werkzeug.security import generate_password_hash, check_password_hash
import threading

app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///users.db'
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
db = SQLAlchemy(app)

class User(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(80), unique=True, nullable=False)
    password = db.Column(db.String(120), nullable=False)

class UserSchema(Schema):
    id = fields.Int(dump_only=True)
    username = fields.Str(required=True, validate=validate.Length(min=1, max=80))
    password = fields.Str(required=True, validate=validate.Length(min=6))

user_schema = UserSchema()
users_schema = UserSchema(many=True)

with app.app_context():
    db.create_all()

@app.route('/')
def home():
    return "Welcome to the Security Testing Demo!"

@app.route('/users', methods=['GET'])
def get_users():
    try:
        users = User.query.all()
        result =  users_schema.dump(users)
        return jsonify(result)
    except Exception as e:
        return jsonify({"message": "An error occurred while fetching users", "error": str(e)}), 500


@app.route('/user/<int:id>', methods=['GET'])
def get_user(id):
    user = User.query.get(id)
    if user:
      result = user_schema.dump(user)
      return jsonify(result)
    return jsonify({"message": "User not found"}), 404

@app.route('/user', methods=['POST'])
def add_user():
    data = request.get_json()
    errors = user_schema.validate(data)
    if errors:
        return jsonify(errors), 400

    hashed_password = generate_password_hash(data['password'])
    new_user = User(username=data['username'], password=hashed_password)
    db.session.add(new_user)
    db.session.commit()
    return jsonify({"message": "User added successfully"}), 201

@app.route('/user/<int:id>', methods=['PUT'])
def update_user(id):
    data = request.get_json()
    errors = user_schema.validate(data)
    if errors:
        return jsonify(errors), 400

    user = User.query.get(id)
    if user:
        user.username = data['username']
        user.password = generate_password_hash(data['password'])
        db.session.commit()
        return jsonify({"message": "User updated successfully"})
    return jsonify({"message": "User not found"}), 404

@app.route('/user/<int:id>', methods=['DELETE'])
def delete_user(id):
    user = User.query.get(id)
    if user:
        db.session.delete(user)
        db.session.commit()
        return jsonify({"message": "User deleted successfully"})
    return jsonify({"message": "User not found"}), 404

threading.Thread(target=app.run, kwargs={'host':'0.0.0.0','port':5000}).start()


 * Serving Flask app '__main__'


In [21]:
!curl -X POST http://127.0.0.1:5000/user -H "Content-Type: application/json" -d '{"username":"testuser","password":"securepassword"}'

<!doctype html>
<html lang=en>
<title>400 Bad Request</title>
<h1>Bad Request</h1>
<p>The browser (or proxy) sent a request that this server could not understand.</p>


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   212  100   167  100    45  17056   4596 --:--:-- --:--:-- --:--:-- 23555


In [22]:
!curl -X GET http://127.0.0.1:5000/users

[]


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100     3  100     3    0     0    392      0 --:--:-- --:--:-- --:--:--   428


In [23]:
!curl -X GET http://localhost:5000/user/1

{"message":"User not found"}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100    29  100    29    0     0    132      0 --:--:-- --:--:-- --:--:--   133


In [18]:
!curl -X PUT http://localhost:5000/user/1 -H "Content-Type: application/json" -d '{"username":"testuser","password":"newsecurepassword"}'

<!doctype html>
<html lang=en>
<title>400 Bad Request</title>
<h1>Bad Request</h1>
<p>The browser (or proxy) sent a request that this server could not understand.</p>


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   215  100   167  100    48   7141   2052 --:--:-- --:--:-- --:--:--  9347


In [19]:
!curl -X DELETE http://localhost:5000/user/1

{"message":"User not found"}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100    29  100    29    0     0    135      0 --:--:-- --:--:-- --:--:--   135
